In [1]:
# **installing dependencies** 

In [3]:
!pip install tensorflow==2.10
!pip install gym
!pip install keras
!pip install keras-rl2

  Created wheel for keras-rl: filename=keras_rl-0.4.2-py3-none-any.whl size=48380 sha256=c259db02f7f7bfbd244be88f318074855c4a2cd0c8165bb6fb370b6845456bd5
  Stored in directory: c:\users\niloofar\appdata\local\pip\cache\wheels\44\fb\c7\96ef7b9a9ded58c3300148907fd398167f44965686e7d04e0f
Successfully built keras-rl


In [ ]:
## test random envirenment with OpenAI Gym

In [4]:
import gym
import random

In [6]:
#setup envirenment
env=gym.make('Acrobot-v1')

In [8]:
# take number of all the space available in our envienment
states=env.observation_space.shape[0]

In [9]:
# extract number of actions
actions=env.action_space.n

In [10]:
print(states)
print(actions)

6
3


In [15]:
# random movments of acrobate 
episodes=10
for episodes in range(1,episodes+1):
        env.reset()
        done=False
        score=0
        while not done:
            env.render()
            action=random.choice([-1,0,1])
            n_state, reward, done, info =env.step(action)
            score+= reward
        print('Episode:{},Score:{}'.format(episodes,score))

Episode:1,Score:-500.0
Episode:2,Score:-500.0
Episode:3,Score:-500.0
Episode:4,Score:-500.0
Episode:5,Score:-500.0
Episode:6,Score:-500.0
Episode:7,Score:-500.0
Episode:8,Score:-500.0
Episode:9,Score:-500.0
Episode:10,Score:-500.0


In [ ]:
# Create a deep learning model with keras

In [45]:
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam

In [81]:
def buildmodel(actions,states):
    model= keras.Sequential(
    [
        Flatten(input_shape=(1,states)),
        Dense(24, activation='relu'),
        Dense(24, activation='relu'),
        Dense(actions,activation='linear')
    ])
    return model

In [82]:
model=buildmodel(actions,states)

In [83]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 6)                 0         
                                                                 
 dense_20 (Dense)            (None, 24)                168       
                                                                 
 dense_21 (Dense)            (None, 24)                600       
                                                                 
 dense_22 (Dense)            (None, 3)                 75        
                                                                 
Total params: 843
Trainable params: 843
Non-trainable params: 0
_________________________________________________________________


In [74]:
#Build Agent with Keras-RL

In [84]:
from rl.agents import DQNAgent #deep-Q-learning
from rl.policy import BoltzmannQPolicy 
from rl.memory import SequentialMemory

In [85]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [86]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-2), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: -0.9975
32 episodes - episode_reward: -309.344 [-500.000, -143.000] - loss: 0.913 - mae: 19.166 - mean_q: -28.078

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: -0.9946
55 episodes - episode_reward: -180.436 [-500.000, -121.000] - loss: 1.531 - mae: 26.521 - mean_q: -38.567

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: -0.9951
50 episodes - episode_reward: -197.740 [-500.000, -123.000] - loss: 1.326 - mae: 25.930 - mean_q: -37.731

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: -0.9948
52 episodes - episode_reward: -194.115 [-331.000, -124.000] - loss: 1.560 - mae: 26.602 - mean_q: -38.680

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 99s 10ms/step

In [89]:
scores=dqn.test(env,nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: -84.000, steps: 85
Episode 2: reward: -75.000, steps: 76
Episode 3: reward: -121.000, steps: 122
Episode 4: reward: -90.000, steps: 91
Episode 5: reward: -77.000, steps: 78
Episode 6: reward: -75.000, steps: 76
Episode 7: reward: -79.000, steps: 80
Episode 8: reward: -98.000, steps: 99
Episode 9: reward: -86.000, steps: 87
Episode 10: reward: -116.000, steps: 117


In [90]:
_=dqn.test(env,nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: -111.000, steps: 112
Episode 2: reward: -78.000, steps: 79
Episode 3: reward: -76.000, steps: 77
Episode 4: reward: -103.000, steps: 104
Episode 5: reward: -114.000, steps: 115
Episode 6: reward: -99.000, steps: 100
Episode 7: reward: -76.000, steps: 77
Episode 8: reward: -94.000, steps: 95
Episode 9: reward: -93.000, steps: 94
Episode 10: reward: -92.000, steps: 93


In [ ]:
#Save Weights

In [58]:
dqn.save_weights('Acrobat_dqn_weights.h5f', overwrite=True)

In [ ]:
# delete everything

In [ ]:
del model
del dqn
del env

In [ ]:
#reload model

In [64]:
env=gym.make('Acrobot-v1')
actions=env.action_space.n
states=env.observation_space.shape[0]
model=buildmodel(actions,states)
dqn=build_agent(model, actions)
dqn.compile(Adam(lr=1e-2),metrics=['mae'])

In [65]:
dqn.load_weights('Acrobat_dqn_weights.h5f')

In [66]:
_=dqn.test(env,nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: -91.000, steps: 92
Episode 2: reward: -77.000, steps: 78
Episode 3: reward: -77.000, steps: 78
Episode 4: reward: -76.000, steps: 77
Episode 5: reward: -79.000, steps: 80
Episode 6: reward: -85.000, steps: 86
Episode 7: reward: -88.000, steps: 89
Episode 8: reward: -85.000, steps: 86
Episode 9: reward: -79.000, steps: 80
Episode 10: reward: -76.000, steps: 77
